In [125]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from MLFM_GCP.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from MLFM_GCP.circuits.cp_fraction import cp_fraction
from MLFM_GCP.circuits.QAOA import QAOA_random
from MLFM_GCP.partitioning.FM.FM_main import *
from MLFM_GCP.partitioning.FM.multilevel_FM_bench import *

from qiskit.circuit.library import QFT, QuantumVolume


detailed_filename = "benchmark_results_MLFM_comparison.json"

exploration = 'explore'
exploration = 'exploit'

if exploration == 'explore':
    stochastic = True
else:
    stochastic = False

# if os.path.exists(detailed_filename):
#     with open(detailed_filename, "r") as f:
#         detailed_results = json.load(f)
# else:

detailed_results = []

sizes = [64]

passes_per_level = 10
fractions = [0.5]
num_partitions_list = [2]

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = num_partitions_list[i]

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
    for fraction in fractions:
        
        # Collect data for computing means across 10 iterations
        iteration_data = []
        
        for iteration in range(1):
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            circuit = cp_fraction(num_qubits, num_qubits, fraction)

            # circuit = QFT(num_qubits, do_swaps=False)
            # circuit = QuantumVolume(num_qubits, num_qubits, seed=0)
            # circuit = QAOA_random(num_qubits, prob = 0.5, reps =1)
            circuit = transpile(circuit, basis_gates=['cp', 'u'])
            base_graph = QuantumCircuitHyperGraph(circuit, group_gates= True, anti_diag = True)
            depth = base_graph.depth
            initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
            num_levels = int(np.ceil(np.log2(depth)))
            print(num_levels)

            limit = num_qubits * depth
            # -------------------------
            # 2. Fine-grained partitioning
            # -------------------------
            graph_list = [base_graph]
            mapping_list = [{i : set([i]) for i in range(depth)}]

            assignment_list_f, cost_list_f, time_list_f, pass_time_list_f, pass_cost_list_f = multilevel_FM_bench(graph_list,
                                                                    mapping_list,
                                                                    initial_assignment,
                                                                    qpu_info,
                                                                    limit=limit,
                                                                    pass_list=[passes_per_level * (num_levels + 1)],
                                                                    stochastic=stochastic,
                                                                    lock_nodes=False,
                                                                    log=False,
                                                                    add_initial=False,
                                                                    costs=None)
            total_time_f = sum(time_list_f)
            min_cost_f = min(cost_list_f)
            # -------------------------
            # 3. Window-based refinement
            # -------------------------

            assignment_list_w, cost_list_w, time_list_w, pass_time_list_w, pass_cost_list_w = MLFM_window_bench(base_graph, 
                num_levels=num_levels,
                initial_assignment=initial_assignment,  
                qpu_info=qpu_info, 
                limit=limit, 
                pass_list=[passes_per_level]*(num_levels+1), 
                stochastic=stochastic, 
                lock_nodes=False,
                log = False,
                add_initial = False,
                costs = None)
            
            total_time_w = sum(time_list_w)
            min_cost_w = min(cost_list_w)

            
            # -------------------------
            # 4. Block refinement
            # -------------------------
            assignment_list_b, cost_list_b, time_list_b, pass_time_list_b, pass_cost_list_b = MLFM_blocks_bench(base_graph,
                                            num_levels=num_levels,
                                            initial_assignment=initial_assignment,  
                                            qpu_info=qpu_info, 
                                            limit=limit, 
                                            pass_list=[passes_per_level] * (num_levels + 1), 
                                            stochastic=stochastic, 
                                            lock_nodes=False,
                                            log = False,
                                            add_initial = False,
                                            costs = None,
                                            full = False)
            
            total_time_b = sum(time_list_b)
            min_cost_b = min(cost_list_b)
            
            # -------------------------
            # 5. Recursive refinement
            # -------------------------
            assignment_list_r, cost_list_r, time_list_r, pass_time_list_r, pass_cost_list_r = MLFM_recursive_bench(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = limit, 
                                        pass_list = [passes_per_level]*(num_levels+1), 
                                        stochastic=stochastic, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            print("Min cost f: ", min_cost_f)
            print("Min cost w: ", min_cost_w)
            print("Min cost b: ", min_cost_b)
            print("Min cost r: ", min_cost_r)

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": fraction,
                "pass_cost_list_f" : pass_cost_list_f,
                "pass_cost_list_w" : pass_cost_list_w,
                "pass_cost_list_b" : pass_cost_list_b,
                "pass_cost_list_r" : pass_cost_list_r,
                "pass_time_list_f" : pass_time_list_f,
                "pass_time_list_w" : pass_time_list_w,
                "pass_time_list_b" : pass_time_list_b,
                "pass_time_list_r" : pass_time_list_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        

print("Benchmarking completed. Detailed results saved to", detailed_filename)

6
Number of levels: 7


KeyboardInterrupt: 

In [120]:
# with open(detailed_filename, "r") as f:

#     data = json.load(f)

data = detailed_results

print(data)

for entry in data:
    cost_list_f = entry["pass_cost_list_f"]
    cost_list_w = entry["pass_cost_list_w"]
    cost_list_b = entry["pass_cost_list_b"]
    cost_list_r = entry["pass_cost_list_r"]
    print(len(cost_list_f), len(cost_list_w), len(cost_list_b), len(cost_list_r))


[{'num_qubits': 128, 'num_partitions': 2, 'fraction': 0.5, 'pass_cost_list_f': [[1275, 1274, 1273, 1272, 1271, 1270, 1270, 1268, 1267, 1267, 1267, 1267, 1267, 1266, 1266, 1266, 1266, 1265, 1265, 1265, 1264, 1263, 1262, 1262, 1262, 1262, 1262, 1262, 1261, 1261, 1260, 1259, 1259, 1258, 1258, 1258, 1258, 1258, 1258, 1258]], 'pass_cost_list_w': [[1161, 1125, 1106, 1101, 1101], [1094, 1091, 1090, 1090, 1090], [1087, 1087, 1087, 1086, 1084], [1077, 1077, 1076, 1076, 1076], [1064, 1062, 1061, 1061, 1060], [1056, 1054, 1054, 1052, 1052], [1049, 1047, 1047, 1046, 1046], [1045, 1045, 1045, 1044, 1044]], 'pass_cost_list_b': [[1150, 1099, 1086, 1077, 1078], [1058, 1045, 1039, 1033, 1033], [1026, 1026, 1026, 1026, 1026], [1023, 1023, 1022, 1022, 1022], [1018, 1018, 1018, 1018, 1017], [1017, 1016, 1015, 1014, 1014], [1013, 1013, 1012, 1011, 1011], [1011, 1011, 1011, 1011, 1011]], 'pass_cost_list_r': [[1148, 1096, 1082, 1079, 1075], [1045, 1037, 1038, 1037, 1037], [1015, 1005, 999, 995, 989], [974, 9

In [121]:
for entry in data:
    time_list_f = copy.deepcopy(entry['pass_time_list_f'])
    time_list_w = copy.deepcopy(entry['pass_time_list_w'])
    time_list_b = copy.deepcopy(entry['pass_time_list_b'])
    time_list_r = copy.deepcopy(entry['pass_time_list_r'])
    counter = 0

    cumulative_time_f = 0
    cumulative_time_w = 0
    cumulative_time_b = 0
    cumulative_time_r = 0
    
    for i in range(len(time_list_w)):
        for j in range(len(time_list_w[i])):
            print(i,j)
            prev_time_f = time_list_f[0][counter] 
            prev_time_w = time_list_w[i][j]
            prev_time_b = time_list_b[i][j]
            prev_time_r = time_list_r[i][j]

            cumulative_time_f += prev_time_f
            cumulative_time_w += prev_time_w
            cumulative_time_b += prev_time_b
            cumulative_time_r += prev_time_r

            time_list_f[0][counter] = cumulative_time_f
            time_list_w[i][j] = cumulative_time_w
            time_list_b[i][j] = cumulative_time_b
            time_list_r[i][j] = cumulative_time_r

            counter += 1
    
    entry['cumulative_time_list_f'] = time_list_f
    entry['cumulative_time_list_w'] = time_list_w
    entry['cumulative_time_list_b'] = time_list_b
    entry['cumulative_time_list_r'] = time_list_r


with open(detailed_filename, "w") as f:
    json.dump(data, f, indent=2)


0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
7 0
7 1
7 2
7 3
7 4


In [122]:
import matplotlib.pyplot as plt
import itertools
data = detailed_results
def flatten_pass_lists(nested_list):

    return list(itertools.chain.from_iterable(nested_list))
with open("dat_files/coarseners_256_2part_cost.dat", "w") as f:
    f.write("pass f w b r\n")
    pass_cost_list_f = flatten_pass_lists(data[0]['pass_cost_list_f'])
    pass_cost_list_w = flatten_pass_lists(data[0]['pass_cost_list_w'])
    pass_cost_list_b = flatten_pass_lists(data[0]['pass_cost_list_b'])
    pass_cost_list_r = flatten_pass_lists(data[0]['pass_cost_list_r'])
    for i in range(len(pass_cost_list_f)):
        f.write(f"{i}" + " " + str(pass_cost_list_f[i]) + " " + str(pass_cost_list_w[i]) + " " + str(pass_cost_list_b[i]) + " " + str(pass_cost_list_r[i]) + "\n")
    
with open("dat_files/coarseners_256_2part_time.dat", "w") as f:
    f.write("pass f w b r\n")
    cumulative_time_list_f = flatten_pass_lists(data[0]['cumulative_time_list_f'])
    cumulative_time_list_w = flatten_pass_lists(data[0]['cumulative_time_list_w'])
    cumulative_time_list_b = flatten_pass_lists(data[0]['cumulative_time_list_b'])
    cumulative_time_list_r = flatten_pass_lists(data[0]['cumulative_time_list_r'])
    for i in range(len(cumulative_time_list_f)):
        f.write(f"{i}" + " " + str(cumulative_time_list_f[i]) + " " + str(cumulative_time_list_w[i]) + " " + str(cumulative_time_list_b[i]) + " " + str(cumulative_time_list_r[i]) + "\n")
    

In [123]:
import json
import itertools
import pandas as pd
import matplotlib.pyplot as plt

def flatten_pass_lists(nested_list):
    """
    Flatten a list of lists into a single list.
    E.g. [[1,2,3],[4,5],[6,7,8]] -> [1,2,3,4,5,6,7,8].
    """
    return list(itertools.chain.from_iterable(nested_list))

# 1) Load the JSON data from file (which is a list of dicts).
with open("benchmark_results_MLFM_comparison.json", "r") as f:
    data_list = json.load(f)  # data_list is now a Python list of dicts

# 2) Create a “long” list of records, one row per (method, pass_number).
#    We’ll store them later in a pandas DataFrame.
records = []

# We know the methods are stored in these keys:
method_keys = {
    "f": "pass_cost_list_f",
    "w": "pass_cost_list_w",
    "b": "pass_cost_list_b",
    "r": "pass_cost_list_r"
}

for entry in data_list:
    num_qubits = entry["num_qubits"]
    num_partitions = entry["num_partitions"]
    fraction = entry["fraction"]
    
    # For each method (f, w, b, r), flatten the pass costs into a single list
    for method_label, json_key in method_keys.items():
        if json_key not in entry:
            # If the JSON doesn't have that key (unlikely), skip it
            continue
        
        nested_pass_data = entry[json_key]  # e.g. pass_cost_list_w
        flattened_costs = flatten_pass_lists(nested_pass_data)
        
        # Each entry in flattened_costs is a cost at pass i.
        # We'll call pass i from 1..N to be consistent.
        for i, cost in enumerate(flattened_costs):
            # Build a dict record for this row
            record = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": fraction,
                "method": method_label,
                "pass": i + 1,
                "cost": cost
            }
            records.append(record)

# 3) Convert the big “records” list into a pandas DataFrame
df = pd.DataFrame(records)

# 4) If you have multiple identical (num_qubits, num_partitions, fraction) sets
#    and you want to average across them, group accordingly.
#    For example, group by (method, pass) and also by (num_qubits, num_partitions, fraction)
group_cols = ["num_qubits", "num_partitions", "fraction", "method", "pass"]
grouped = df.groupby(group_cols).agg(
    mean_cost=("cost", "mean"),
    std_cost=("cost", "std")  # optional, for error bars
).reset_index()

# 5) Now pick which combination of (num_qubits, num_partitions, fraction) you want to plot.
#    If you have only 1 unique set, that’s easy. Otherwise, you can loop over them.
unique_combos = grouped[["num_qubits", "num_partitions", "fraction"]].drop_duplicates()

for _, combo_row in unique_combos.iterrows():
    nq = combo_row["num_qubits"]
    nparts = combo_row["num_partitions"]
    frac = combo_row["fraction"]
    
    # Filter just these qubits/partitions/fraction
    subset = grouped[
        (grouped["num_qubits"] == nq)
        & (grouped["num_partitions"] == nparts)
        & (grouped["fraction"] == frac)
    ]
    
    # We'll pivot or just iterate by “method” to plot each line
    fig, ax = plt.subplots(figsize=(8,4))
    
    methods = subset["method"].unique()
    
    for m in methods:
        sub2 = subset[subset["method"] == m].sort_values("pass")
        
        ax.errorbar(
            sub2["pass"],
            sub2["mean_cost"],
            yerr=sub2["std_cost"],          # comment this out if no error bars
            label=f"Method {m.upper()}",    # e.g. F, W, B, R
            marker='o', linestyle='-'
        )
    
    ax.set_xlabel("Pass number")
    ax.set_ylabel("Cost")
    ax.set_title(f"{nq} qubits, {nparts} partitions, fraction={frac}")
    ax.legend()
    plt.tight_layout()
    
    # Show or save the figure:
    # plt.show()
    plt.savefig(f"plot_cost_{nq}q_{nparts}p_frac{frac}_{exploration}.png")
    plt.close(fig)

In [124]:
import json
import itertools
import pandas as pd
import matplotlib.pyplot as plt

def flatten_pass_lists(nested_list):
    """
    Flatten a list of lists into a single list.
    E.g. [[1,2,3],[4,5],[6,7,8]] -> [1,2,3,4,5,6,7,8].
    """
    return list(itertools.chain.from_iterable(nested_list))

# 1) Load the JSON data from file (which is a list of dicts).
with open("benchmark_results_MLFM_comparison.json", "r") as f:
    data_list = json.load(f)  # data_list is now a Python list of dicts

# 2) Create a “long” list of records, one row per (method, pass_number).
#    We’ll store them later in a pandas DataFrame.
records = []

# We know the methods are stored in these keys:
method_keys = {
    "f": "cumulative_time_list_f",
    "w": "cumulative_time_list_w",
    "b": "cumulative_time_list_b",
    "r": "cumulative_time_list_r"
}

for entry in data_list:
    num_qubits = entry["num_qubits"]
    num_partitions = entry["num_partitions"]
    fraction = entry["fraction"]
    
    # For each method (f, w, b, r), flatten the pass costs into a single list
    for method_label, json_key in method_keys.items():
        if json_key not in entry:
            # If the JSON doesn't have that key (unlikely), skip it
            continue
        
        nested_pass_data = entry[json_key]  # e.g. pass_cost_list_w
        flattened_costs = flatten_pass_lists(nested_pass_data)
        
        # Each entry in flattened_costs is a cost at pass i.
        # We'll call pass i from 1..N to be consistent.
        for i, cost in enumerate(flattened_costs):
            # Build a dict record for this row
            record = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": fraction,
                "method": method_label,
                "pass": i + 1,
                "cost": cost
            }
            records.append(record)

# 3) Convert the big “records” list into a pandas DataFrame
df = pd.DataFrame(records)

# 4) If you have multiple identical (num_qubits, num_partitions, fraction) sets
#    and you want to average across them, group accordingly.
#    For example, group by (method, pass) and also by (num_qubits, num_partitions, fraction)
group_cols = ["num_qubits", "num_partitions", "fraction", "method", "pass"]
grouped = df.groupby(group_cols).agg(
    mean_cost=("cost", "mean"),
    std_cost=("cost", "std")  # optional, for error bars
).reset_index()

# 5) Now pick which combination of (num_qubits, num_partitions, fraction) you want to plot.
#    If you have only 1 unique set, that’s easy. Otherwise, you can loop over them.
unique_combos = grouped[["num_qubits", "num_partitions", "fraction"]].drop_duplicates()

for _, combo_row in unique_combos.iterrows():
    nq = combo_row["num_qubits"]
    nparts = combo_row["num_partitions"]
    frac = combo_row["fraction"]
    
    # Filter just these qubits/partitions/fraction
    subset = grouped[
        (grouped["num_qubits"] == nq)
        & (grouped["num_partitions"] == nparts)
        & (grouped["fraction"] == frac)
    ]
    
    # We'll pivot or just iterate by “method” to plot each line
    fig, ax = plt.subplots(figsize=(8,4))
    
    methods = subset["method"].unique()
    
    for m in methods:
        sub2 = subset[subset["method"] == m].sort_values("pass")
        
        ax.errorbar(
            sub2["pass"],
            sub2["mean_cost"],
            yerr=sub2["std_cost"],          # comment this out if no error bars
            label=f"Method {m.upper()}",    # e.g. F, W, B, R
            marker='o', linestyle='-'
        )
    
    ax.set_xlabel("Pass number")
    ax.set_ylabel("Cost")
    ax.set_title(f"{nq} qubits, {nparts} partitions, fraction={frac}")
    ax.legend()
    plt.tight_layout()
    
    # Show or save the figure:
    # plt.show()
    plt.savefig(f"_plot_time_{nq}q_{nparts}p_frac{frac}_{exploration}.png")
    plt.close(fig)